In [1]:
import os
import math
import numpy as np
import pandas as pd
import keras

!pip install xlrd

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler

print("Keras version", keras.__version__)
print("Numpy version", np.__version__)

Using TensorFlow backend.


Keras version 2.1.5
Numpy version 1.14.1


In [2]:
# upload test excel file
from google.colab import files
uploaded = files.upload()
for xl in uploaded.keys():
  print('Uploaded Test file "{name}" with length {length} bytes'.format(
         name=xl, length=len(uploaded[xl])))
  excel_file = xl
  break

print(excel_file)
df_xl = pd.read_excel(excel_file)
df      = df_xl.transpose()
display(df[:5])

Saving test01.xlsx to test01 (1).xlsx
Uploaded Test file "test01.xlsx" with length 9982 bytes
test01.xlsx


,0,1,2,3,4,5,6
Input 1,1,5,10,14,23,24,38
Input 2,2,4,9,24,25,27,45
Input 3,3,1,5,11,32,40,45
Input 4,4,19,36,39,41,46,51
Input 5,5,10,11,19,41,50,54


In [3]:
# upload neural network model file.
# It'll be present in <Downloads> dir with name "nn_model.h5"
from keras.models import load_model
uploaded = files.upload()
for mdl in uploaded.keys():
  print('Uploaded Test file "{name}" with length {length} bytes'.format(
         name=mdl, length=len(uploaded[mdl])))
  nn_model = mdl
  break

model = load_model(nn_model)

Saving nn_model.h5 to nn_model (4).h5
Uploaded Test file "nn_model.h5" with length 27080 bytes


In [4]:
batch_size   = 1
look_back    = 2
n_features   = 7

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), :]
		dataX.append(a)
		dataY.append(dataset[i + look_back, :])
	return np.array(dataX), np.array(dataY)


# normalize dataset
dataset = df.values
dataset = dataset.astype('float32')
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

if ( n_features != dataset.shape[1] ):
  raise ValueError("Excel file format does not match with trained model.")

test = dataset[35:,:]

# reshape into X=t and Y=t+1

testX, testY = create_dataset(test, look_back)
testPredict = model.predict(testX, batch_size=batch_size)

normalizedTestPrediction =  scaler.inverse_transform(testPredict)
#normalizedTest           =  scaler.inverse_transform(test[3:])


print("Prediction:\n", normalizedTestPrediction[1:,:].astype(dtype=np.int32))

Prediction:
 [[28 10 18 25 27 37 47]
 [36 16 25 31 33 44 52]]


In [9]:
print("Prediction for last 6 columns:\n",
      normalizedTestPrediction[:,1:].astype(dtype=np.int32))

Prediction for last 6 columns:
 [[10 18 25 27 37 47]
 [16 25 31 33 44 52]]
